In [1]:
import pandas as pd
import numpy as np

In [2]:
mathclass = pd.read_csv("student-mat.csv",delimiter=";") #math course csv
portclass = pd.read_csv("student-por.csv",delimiter=";") #portuguese language course csv

print('Math class data points: {}'.format(len(mathclass)))
print('Portuguese class data points: {}'.format(len(portclass)))
print('Total data points: {}'.format(len(mathclass)+len(portclass)))

Math class data points: 395
Portuguese class data points: 649
Total data points: 1044


In [3]:
#define lists of column names
all_courseindependent = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian', \
                         'traveltime','studytime','failures','schoolsup','famsup','activities','nursery','higher','internet', \
                         'romantic','famrel','freetime','goout','Dalc','Walc','health']
all_ = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian','traveltime', \
       'studytime','failures','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic','famrel', \
       'freetime','goout','Dalc','Walc','health','absences','G1','G2','G3']
student_merge = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','nursery','internet']
math_columns = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian', \
                'traveltime','studytime','failures','schoolsup','famsup','paid_math','activities','nursery','higher', \
                'internet','romantic','famrel','freetime','goout','Dalc','Walc','health','absences_math','G1_math', \
                'G2_math','G3_math','math_only']
port_columns = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian', \
                'traveltime','studytime','failures','schoolsup','famsup','paid_port','activities','nursery','higher', \
                'internet','romantic','famrel','freetime','goout','Dalc','Walc','health','absences_port','G1_port', \
                'G2_port','G3_port','port_only']

## Choose columns to merge datasets on

The data source states that there are 382 students that belong to both classes (math and portuguese).  Included with the datasets was an example file of how to find the students which are included both datasets.  The example file merged the two datasets on the following columns: school, sex, age, address, famsize, Pstatus, Medu, Fedu, Mjob, Fjob, reason, nursery, and internet.

In [4]:
bothclass = pd.concat([portclass, mathclass])
print('Number of students in both classes: {}'.format(bothclass.duplicated(keep='first',subset=student_merge).sum()))

Number of students in both classes: 382


Before going forward with the list of columns provided by the source, we will test whether the list includes enough columns to avoid incorrectly labeling data as a duplicate.

In [5]:
#determine whether there are duplicates in the individual datasets
print('Number of duplicates found in Portuguese class dataset: {}'. \
      format(portclass.duplicated(keep='first',subset=student_merge).sum()))
print('Number of duplicates found in math class dataset: {}'. \
      format(mathclass.duplicated(keep='first',subset=student_merge).sum()))

Number of duplicates found in Portuguese class dataset: 12
Number of duplicates found in math class dataset: 4


Since duplicates appear to be present in the individual datasets when the list of columns provided by the source was used to find duplicates, we will determine if they are actually duplicates by comparing all of the columns in each dataframe.

In [6]:
#determine whether there are duplicates in the individual datasets
print('Number of duplicates found in Portuguese class dataset: {}'. \
      format(portclass.duplicated(keep='first',subset=all_).sum()))
print('Number of duplicates found in math class dataset: {}'. \
      format(mathclass.duplicated(keep='first',subset=all_).sum()))

Number of duplicates found in Portuguese class dataset: 0
Number of duplicates found in math class dataset: 0


When we search for duplicates using all of the columns in each dataframe, no duplicates are found.  Thus, we can assume that the list of columns provided by the source for merging the dataframes is not suffucient to find true duplicates between the two datasets.  To determine whether the set of all columns will find duplicates betwwen the two datasets, we search for duplicates in the concatenated dataframe, considering all columns.

In [7]:
print('Number of students in both classes: {}'.format(bothclass.duplicated(keep='first',subset=all_).sum()))

Number of students in both classes: 0


Using all of the columns to find duplicates between the datasets resulted in 0 duplicates being found.  Thus, there may be some columns which are based on the class which the student's data was aquired in.  From information about the columns, it appears that the following columns are class-specific:

* paid: extra paid classes within the course subject
* absences: number of school absences
* G1: first period grade
* G2: second period grade
* G3: third period grade

The class-specific columns were removed from the list of columns to consider when finding duplicates.  

In [8]:
#determine whether there are duplicates in the individual datasets
print('Number of duplicates found in Portuguese class dataset: {}'. \
      format(portclass.duplicated(keep='first',subset=all_courseindependent).sum()))
print('Number of duplicates found in math class dataset: {}'. \
      format(mathclass.duplicated(keep='first',subset=all_courseindependent).sum()))

Number of duplicates found in Portuguese class dataset: 2
Number of duplicates found in math class dataset: 0


In [9]:
print('Number of students in both classes: {}'.format(bothclass.duplicated(keep='first',subset=all_courseindependent).sum()))

Number of students in both classes: 322


When all columns except for the class-specific columns were considered, only 2 duplicates were found in the Portuguese class dataset and no duplicates were found in the math class.

In order to verify that all class-independent columns should be used to find duplicates between the two datasets, we added or omitted one column at a time from the list.  Whenever one of the five class-specific columns were included in the list, the number of duplicates reduced significantly.  On the other hand, when one class-independent column was omitted from the list, the number of duplicates did not change.

# How to handle 2 duplicates that show up in Portuguese class when class-independent columns used for merging?

Exclude from analysis, or keep even though they throw off class_num?

## If choose to exclude from analysis

In [10]:
portclass_nodup = portclass.drop_duplicates(subset=all_courseindependent,keep='first')
mathclass_nodup = mathclass.drop_duplicates(subset=all_courseindependent,keep='first')

To prepare to merge mathclass and portclass dataframes, we will create columns for the class-specific columns which spectify which class the values in the column are from.

In [11]:
mathclass['paid_math'] = mathclass['paid']
mathclass['absences_math'] = mathclass['absences']
mathclass['G1_math'] = mathclass['G1']
mathclass['G2_math'] = mathclass['G2']
mathclass['G3_math'] = mathclass['G3']
mathclass['math_only'] = 1

portclass['paid_port'] = portclass['paid']
portclass['absences_port'] = portclass['absences']
portclass['G1_port'] = portclass['G1']
portclass['G2_port'] = portclass['G2']
portclass['G3_port'] = portclass['G3']
portclass['port_only'] = 1

mathclass_merge = mathclass[math_columns]
portclass_merge = portclass[port_columns]

#commment out if want to include duplicate Portuguese students
portclass_nodup = portclass_merge.drop_duplicates(subset=all_courseindependent,keep='first')
mathclass_nodup = mathclass_merge.drop_duplicates(subset=all_courseindependent,keep='first')

Merge mathclass and portclass dataframes

In [12]:
bothclass_merge = pd.merge(portclass_nodup, mathclass_nodup, how='outer', on=all_courseindependent)
print(bothclass_merge.columns.tolist())

bothclass_merge

['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid_port', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences_port', 'G1_port', 'G2_port', 'G3_port', 'port_only', 'paid_math', 'absences_math', 'G1_math', 'G2_math', 'G3_math', 'math_only']


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,G1_port,G2_port,G3_port,port_only,paid_math,absences_math,G1_math,G2_math,G3_math,math_only
0,GP,F,18.0,U,GT3,A,4.0,4.0,at_home,teacher,...,0.0,11.0,11.0,1.0,no,6.0,5.0,6.0,6.0,1.0
1,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,9.0,11.0,11.0,1.0,no,4.0,5.0,5.0,6.0,1.0
2,GP,F,15.0,U,LE3,T,1.0,1.0,at_home,other,...,12.0,13.0,12.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,GP,F,15.0,U,GT3,T,4.0,2.0,health,services,...,14.0,14.0,14.0,1.0,yes,2.0,15.0,14.0,15.0,1.0
4,GP,F,16.0,U,GT3,T,3.0,3.0,other,other,...,11.0,13.0,13.0,1.0,yes,4.0,6.0,10.0,10.0,1.0
5,GP,M,16.0,U,LE3,T,4.0,3.0,services,other,...,12.0,12.0,13.0,1.0,yes,10.0,15.0,15.0,15.0,1.0
6,GP,M,16.0,U,LE3,T,2.0,2.0,other,other,...,13.0,12.0,13.0,1.0,no,0.0,12.0,12.0,11.0,1.0
7,GP,F,17.0,U,GT3,A,4.0,4.0,other,teacher,...,10.0,13.0,13.0,1.0,no,6.0,6.0,5.0,6.0,1.0
8,GP,M,15.0,U,LE3,A,3.0,2.0,services,other,...,15.0,16.0,17.0,1.0,yes,0.0,16.0,18.0,19.0,1.0
9,GP,M,15.0,U,GT3,T,3.0,4.0,other,other,...,12.0,12.0,13.0,1.0,yes,0.0,14.0,15.0,15.0,1.0


In [13]:
#check that numbers work out

#merge 2 datasets into single dataframe
bothclass = pd.concat([portclass_nodup, mathclass_nodup])
bothclass = bothclass.reset_index(drop=True)
print('Length of concatenated datasets from both classes: {}'.format(len(bothclass)))

#find duplicates that are in both math and portuguese class datasets
duplicates = bothclass.duplicated(subset=all_courseindependent,keep='last')
print('Number of duplicates: {}'.format(duplicates.sum()))
# Set duplicate values to have class category of 2, or both classes
bothclass.loc[duplicates, 'math_only'] = 0
bothclass.loc[duplicates, 'port_only'] = 0

# Remove other duplicates
bothclass = bothclass.drop_duplicates(keep='first', subset=all_courseindependent)
bothclass = bothclass.reset_index(drop=True)
print('Length of dataset after duplicates removed: {}'.format(len(bothclass)))

print('Length of mathclass dataset: {}'.format(len(mathclass_nodup)))
print('Number of mathclass only in bothclass: {}'.format(len(bothclass[bothclass['math_only']==1])))
print('Length of portclass dataset: {}'.format(len(portclass_nodup)))
print('Number of portclass only in bothclass: {}'.format(len(bothclass[bothclass['port_only']==1])))
print('Number of mathclass AND portclass in bothclass: {}'.format(len(bothclass[bothclass['math_only']==0])))
print('Number of mathclass AND portclass in bothclass: {}'.format(len(bothclass[bothclass['port_only']==0])))

Length of concatenated datasets from both classes: 1042
Number of duplicates: 320
Length of dataset after duplicates removed: 722
Length of mathclass dataset: 395
Number of mathclass only in bothclass: 75
Length of portclass dataset: 647
Number of portclass only in bothclass: 327
Number of mathclass AND portclass in bothclass: 320
Number of mathclass AND portclass in bothclass: 320


In [14]:
print(bothclass.columns.tolist())

bothclass

['Dalc', 'Fedu', 'Fjob', 'G1_math', 'G1_port', 'G2_math', 'G2_port', 'G3_math', 'G3_port', 'Medu', 'Mjob', 'Pstatus', 'Walc', 'absences_math', 'absences_port', 'activities', 'address', 'age', 'failures', 'famrel', 'famsize', 'famsup', 'freetime', 'goout', 'guardian', 'health', 'higher', 'internet', 'math_only', 'nursery', 'paid_math', 'paid_port', 'port_only', 'reason', 'romantic', 'school', 'schoolsup', 'sex', 'studytime', 'traveltime']


,Dalc,Fedu,Fjob,G1_math,G1_port,G2_math,G2_port,G3_math,G3_port,Medu,...,paid_math,paid_port,port_only,reason,romantic,school,schoolsup,sex,studytime,traveltime
0,1,4,teacher,NaN,0.0,NaN,11.0,NaN,11.0,4,...,NaN,no,0.0,course,no,GP,yes,F,2,2
1,1,1,other,NaN,9.0,NaN,11.0,NaN,11.0,1,...,NaN,no,0.0,course,no,GP,no,F,2,1
2,2,1,other,NaN,12.0,NaN,13.0,NaN,12.0,1,...,NaN,no,1.0,other,no,GP,yes,F,2,1
3,1,2,services,NaN,14.0,NaN,14.0,NaN,14.0,4,...,NaN,no,0.0,home,yes,GP,no,F,3,1
4,1,3,other,NaN,11.0,NaN,13.0,NaN,13.0,3,...,NaN,no,0.0,home,no,GP,no,F,2,1
5,1,3,other,NaN,12.0,NaN,12.0,NaN,13.0,4,...,NaN,no,0.0,reputation,no,GP,no,M,2,1
6,1,2,other,NaN,13.0,NaN,12.0,NaN,13.0,2,...,NaN,no,0.0,home,no,GP,no,M,2,1
7,1,4,teacher,NaN,10.0,NaN,13.0,NaN,13.0,4,...,NaN,no,0.0,home,no,GP,yes,F,2,2
8,1,2,other,NaN,15.0,NaN,16.0,NaN,17.0,3,...,NaN,no,0.0,home,no,GP,no,M,2,1
9,1,4,other,NaN,12.0,NaN,12.0,NaN,13.0,3,...,NaN,no,0.0,home,no,GP,no,M,2,1


In [15]:
#data cleanup 

bothclass_merge['school_num'] = bothclass_merge['school'].map({'GP':0, 'MS':1})
bothclass_merge['sex_num'] = bothclass_merge['sex'].map({'F':0, 'M':1})
bothclass_merge['address_num'] = bothclass_merge['address'].map({'U':0, 'R':1})
bothclass_merge['famsize_num'] = bothclass_merge['famsize'].map({'LE3':0, 'GT3':1})
bothclass_merge['Pstatus_num'] = bothclass_merge['Pstatus'].map({'T':0, 'A':1})
bothclass_merge['Mjob_num'] = bothclass_merge['Mjob'].map({'teacher':0, 'health':1, 'services':2, 'at_home':3, 'other':4})
bothclass_merge['Fjob_num'] = bothclass_merge['Fjob'].map({'teacher':0, 'health':1, 'services':2, 'at_home':3, 'other':4})
bothclass_merge['reason_num'] = bothclass_merge['reason'].map({'home':0, 'reputation':1, 'course':2, 'other':3})
bothclass_merge['guardian_num'] = bothclass_merge['guardian'].map({'mother':0, 'father':1, 'other':2})
bothclass_merge['schoolsup_num'] = bothclass_merge['schoolsup'].map({'yes':0, 'no':1})
bothclass_merge['famsup_num'] = bothclass_merge['famsup'].map({'yes':0, 'no':1})
bothclass_merge['activities_num'] = bothclass_merge['activities'].map({'yes':0, 'no':1})
bothclass_merge['nursery_num'] = bothclass_merge['nursery'].map({'yes':0, 'no':1})
bothclass_merge['higher_num'] = bothclass_merge['higher'].map({'yes':0, 'no':1})
bothclass_merge['internet_num'] = bothclass_merge['internet'].map({'yes':0, 'no':1})
bothclass_merge['romantic_num'] = bothclass_merge['romantic'].map({'yes':0, 'no':1})
bothclass_merge['paid_port_num'] = bothclass_merge['paid_port'].map({'yes':0, 'no':1})
bothclass_merge['paid_math_num'] = bothclass_merge['paid_math'].map({'yes':0, 'no':1})

bothclass['school_num'] = bothclass['school'].map({'GP':0, 'MS':1})
bothclass['sex_num'] = bothclass['sex'].map({'F':0, 'M':1})
bothclass['address_num'] = bothclass['address'].map({'U':0, 'R':1})
bothclass['famsize_num'] = bothclass['famsize'].map({'LE3':0, 'GT3':1})
bothclass['Pstatus_num'] = bothclass['Pstatus'].map({'T':0, 'A':1})
bothclass['Mjob_num'] = bothclass['Mjob'].map({'teacher':0, 'health':1, 'services':2, 'at_home':3, 'other':4})
bothclass['Fjob_num'] = bothclass['Fjob'].map({'teacher':0, 'health':1, 'services':2, 'at_home':3, 'other':4})
bothclass['reason_num'] = bothclass['reason'].map({'home':0, 'reputation':1, 'course':2, 'other':3})
bothclass['guardian_num'] = bothclass['guardian'].map({'mother':0, 'father':1, 'other':2})
bothclass['schoolsup_num'] = bothclass['schoolsup'].map({'yes':0, 'no':1})
bothclass['famsup_num'] = bothclass['famsup'].map({'yes':0, 'no':1})
bothclass['activities_num'] = bothclass['activities'].map({'yes':0, 'no':1})
bothclass['nursery_num'] = bothclass['nursery'].map({'yes':0, 'no':1})
bothclass['higher_num'] = bothclass['higher'].map({'yes':0, 'no':1})
bothclass['internet_num'] = bothclass['internet'].map({'yes':0, 'no':1})
bothclass['romantic_num'] = bothclass['romantic'].map({'yes':0, 'no':1})
bothclass['paid_port_num'] = bothclass['paid_port'].map({'yes':0, 'no':1})
bothclass['paid_math_num'] = bothclass['paid_math'].map({'yes':0, 'no':1})

I found 2 methods to combine the datasets.  We can use either one since they result in the same dataframe.

In [16]:
#concatenated dataset
bothclass.to_csv('CleanDatav2.csv', sep=',', na_rep=np.nan)
#merged dataset
bothclass_merge.to_csv('CleanDatav2merge.csv', sep=',', na_rep=np.nan)